In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
import tensorflow_face_conv as myconv
IMGSIZE = 64


def onehot(numlist):
    ''' get one hot return host matrix is len * max+1 demensions'''
    b = np.zeros([len(numlist), max(numlist)+1])
    b[np.arange(len(numlist)), numlist] = 1
    return b.tolist()

def getfileandlabel(filedir):
    ''' get path and host paire and class index to name'''
    dictdir = dict([[name, os.path.join(filedir, name)] \
                    for name in os.listdir(filedir) if os.path.isdir(os.path.join(filedir, name))])
                    #for (path, dirnames, _) in os.walk(filedir) for dirname in dirnames])

    dirnamelist, dirpathlist = dictdir.keys(), dictdir.values()
    indexlist = list(range(len(dirnamelist)))

    return list(zip(dirpathlist, onehot(indexlist))), dict(zip(indexlist, dirnamelist))


C:\Users\long\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def testfromcamera(chkpoint):
    pathlabelpair, indextoname = getfileandlabel('faceImageGray')
    cap = cv2.VideoCapture(0)
    haar = cv2.CascadeClassifier("haarcascades\\haarcascade_frontalface_default.xml")
    Label = np.load('label.npy')
    label = Label.tolist()
    output = myconv.cnnLayer(len(label))
    #predict = tf.equal(tf.argmax(output, 1), tf.argmax(y_data, 1))
    predict = output

    saver = tf.train.Saver()
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, chkpoint)
        

        n = 1
        while 1:
            if (n <= 20000):
                print('It`s processing %s image.' % n)
                # 读帧
                sucess,img=cap.read()
                gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                faces = haar.detectMultiScale(gray_img, 1.3, 5)
                for f_x, f_y, f_w, f_h in faces:
                    face = img[f_y:f_y+f_h, f_x:f_x+f_w]
                    face = cv2.resize(face, (IMGSIZE, IMGSIZE))
                    #could deal with face to train
                    test_x = np.array([face])
                    test_x = test_x.astype(np.float32) / 255.0
                    
                    res = sess.run([predict, tf.argmax(output, 1)],\
                                   feed_dict={myconv.x_data: test_x,\
                                   myconv.keep_prob_5:1.0, myconv.keep_prob_75: 1.0})
                    print(res)

                    cv2.putText(img, indextoname.get('res[1][0]','not exist'), (f_x, f_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 2)  #显示名字
                    img = cv2.rectangle(img, (f_x, f_y), (f_x + f_w, f_y + f_h), (255, 0, 0), 2)
                    n+=1
                cv2.imshow('img', img)
                key = cv2.waitKey(30) & 0xff
                if key == 27:
                    break
            else:
                break
    cap.release()
    cv2.destroyAllWindows()
    
    
if __name__ == '__main__':
    savepath = 'checkpoint/face.ckpt'
    testfromcamera(savepath)

INFO:tensorflow:Restoring parameters from checkpoint/face.ckpt
It`s processing 1 image.
[array([[-0.49951047, -1.455993  , -1.8299428 , -1.2334487 , -0.15548164,
        -0.3065446 ,  0.19717818,  0.38197792, -1.1044888 , -2.297509  ,
         0.86421984]], dtype=float32), array([10], dtype=int64)]
It`s processing 2 image.
[array([[-0.49950978, -1.4559929 , -1.8299425 , -1.2334462 , -0.15547779,
        -0.30655372,  0.19718036,  0.38197464, -1.1044859 , -2.297504  ,
         0.86421055]], dtype=float32), array([10], dtype=int64)]
It`s processing 3 image.
[array([[-0.49950942, -1.4559928 , -1.8299427 , -1.2334483 , -0.15547648,
        -0.3065571 ,  0.19717932,  0.3819764 , -1.104487  , -2.2975068 ,
         0.8642092 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 4 image.
[array([[-0.4995117 , -1.4559906 , -1.829943  , -1.2334522 , -0.15547279,
        -0.30655712,  0.19717771,  0.38197696, -1.1044884 , -2.2975104 ,
         0.86420876]], dtype=float32), array([10], dty

It`s processing 38 image.
[array([[-0.49950707, -1.4559909 , -1.8299372 , -1.2334476 , -0.15546185,
        -0.30657685,  0.19716868,  0.3819679 , -1.1044879 , -2.2975042 ,
         0.8641811 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 39 image.
[array([[-0.49951828, -1.455986  , -1.8299408 , -1.2334385 , -0.15549079,
        -0.30651692,  0.19718057,  0.38198116, -1.1044962 , -2.297508  ,
         0.8642597 ]], dtype=float32), array([10], dtype=int64)]
[array([[-0.49949852, -1.4560032 , -1.8299366 , -1.2334447 , -0.15547389,
        -0.30658633,  0.19716346,  0.3819729 , -1.1044995 , -2.2975059 ,
         0.8641919 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 41 image.
[array([[-0.499521  , -1.4559847 , -1.8299379 , -1.2334433 , -0.15549833,
        -0.3065034 ,  0.19718486,  0.38198632, -1.1044879 , -2.2975159 ,
         0.8642664 ]], dtype=float32), array([10], dtype=int64)]
[array([[-0.4994965 , -1.4559991 , -1.8299409 , -1.2334502 , -0.15547436,


[array([[-0.4994822 , -1.45598   , -1.8299694 , -1.2334467 , -0.1554496 ,
        -0.30660892,  0.19717935,  0.3819497 , -1.1045092 , -2.297487  ,
         0.86414623]], dtype=float32), array([10], dtype=int64)]
[array([[-0.49949622, -1.455992  , -1.8299541 , -1.2334507 , -0.15546757,
        -0.306607  ,  0.19716427,  0.3819641 , -1.1044909 , -2.2974954 ,
         0.86418384]], dtype=float32), array([10], dtype=int64)]
It`s processing 76 image.
[array([[-0.4994992 , -1.4559975 , -1.8299557 , -1.2334479 , -0.15545535,
        -0.3066084 ,  0.19717097,  0.38195837, -1.1044945 , -2.2975004 ,
         0.8641797 ]], dtype=float32), array([10], dtype=int64)]
[array([[-0.4994865 , -1.4559764 , -1.829966  , -1.2334509 , -0.1554434 ,
        -0.30661502,  0.19716948,  0.38194215, -1.1045086 , -2.297481  ,
         0.8641468 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 78 image.
[array([[-0.49948406, -1.455971  , -1.8299654 , -1.2334441 , -0.15544069,
        -0.30662236,  0.19

[array([[-0.49950248, -1.4559946 , -1.8299389 , -1.2334461 , -0.15547374,
        -0.30655164,  0.197168  ,  0.3819757 , -1.1044927 , -2.2975078 ,
         0.8642091 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 112 image.
[array([[-0.49950302, -1.4559938 , -1.8299365 , -1.2334476 , -0.15547347,
        -0.3065484 ,  0.19716978,  0.3819767 , -1.1044931 , -2.2975082 ,
         0.864208  ]], dtype=float32), array([10], dtype=int64)]
It`s processing 113 image.
[array([[-0.49950626, -1.4559941 , -1.829937  , -1.2334483 , -0.15547368,
        -0.30654392,  0.19716913,  0.381979  , -1.1044918 , -2.2975073 ,
         0.8642125 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 114 image.
[array([[-0.49950624, -1.4559951 , -1.8299421 , -1.2334532 , -0.15547252,
        -0.3065421 ,  0.1971671 ,  0.38197532, -1.104491  , -2.297511  ,
         0.86421454]], dtype=float32), array([10], dtype=int64)]
It`s processing 115 image.
[array([[-0.49950534, -1.4559933 , -1.829939

It`s processing 148 image.
[array([[-0.49951234, -1.4559975 , -1.829933  , -1.2334504 , -0.1554695 ,
        -0.3065954 ,  0.19715443,  0.3819543 , -1.1044866 , -2.2974896 ,
         0.86418414]], dtype=float32), array([10], dtype=int64)]
It`s processing 149 image.
[array([[-0.49951106, -1.4560094 , -1.8299816 , -1.2334468 , -0.15547886,
        -0.3065691 ,  0.19718903,  0.38198015, -1.1044878 , -2.297504  ,
         0.86422604]], dtype=float32), array([10], dtype=int64)]
It`s processing 150 image.
It`s processing 150 image.
It`s processing 150 image.
It`s processing 150 image.
It`s processing 150 image.
It`s processing 150 image.
[array([[-0.4995089 , -1.4560112 , -1.8299768 , -1.2334439 , -0.15546927,
        -0.30657712,  0.19719687,  0.38197592, -1.1044861 , -2.2975025 ,
         0.8642268 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 151 image.
[array([[-0.49950677, -1.456004  , -1.8299396 , -1.2334486 , -0.15546149,
        -0.3066101 ,  0.1971542 ,  0.3819456 , 

...

It`s processing 363 image.
[array([[-0.49947575, -1.455984  , -1.8299627 , -1.233438  , -0.155437  ,
        -0.306633  ,  0.1971792 ,  0.38194656, -1.1044912 , -2.2975013 ,
         0.86414635]], dtype=float32), array([10], dtype=int64)]
It`s processing 364 image.
[array([[-0.4994799 , -1.4559827 , -1.8299642 , -1.2334394 , -0.15543678,
        -0.30663013,  0.1971786 ,  0.38194436, -1.1044914 , -2.297504  ,
         0.8641533 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 365 image.
[array([[-0.49947792, -1.4559834 , -1.8299589 , -1.2334408 , -0.15543535,
        -0.30662373,  0.19717088,  0.3819461 , -1.1044972 , -2.2975023 ,
         0.86415094]], dtype=float32), array([10], dtype=int64)]
It`s processing 366 image.
[array([[-0.4994792 , -1.4559817 , -1.8299589 , -1.233441  , -0.15543997,
        -0.30662918,  0.19718093,  0.3819471 , -1.1044911 , -2.2975047 ,
         0.86415446]], dtype=float32), array([10], dtype=int64)]
It`s processing 367 image.
[array([[-0.49948

It`s processing 400 image.
[array([[-0.4994905 , -1.4560153 , -1.8299611 , -1.2334383 , -0.15546653,
        -0.30659717,  0.19715819,  0.3819535 , -1.104513  , -2.297492  ,
         0.86416733]], dtype=float32), array([10], dtype=int64)]
It`s processing 401 image.
[array([[-0.4994895 , -1.4560151 , -1.8299625 , -1.2334379 , -0.15546751,
        -0.3066011 ,  0.19715872,  0.38195488, -1.1045128 , -2.2974894 ,
         0.864165  ]], dtype=float32), array([10], dtype=int64)]
It`s processing 402 image.
[array([[-0.49949247, -1.4560158 , -1.8299602 , -1.2334403 , -0.15546644,
        -0.30660278,  0.19715977,  0.3819539 , -1.1045141 , -2.29749   ,
         0.8641659 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 403 image.
[array([[-0.49949217, -1.4560175 , -1.8299595 , -1.2334397 , -0.15546593,
        -0.30660313,  0.19716093,  0.38195467, -1.1045147 , -2.2974894 ,
         0.8641677 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 404 image.
[array([[-0.49949

[array([[-0.49948117, -1.4559948 , -1.8299567 , -1.2334458 , -0.15546808,
        -0.30658412,  0.19716871,  0.38195723, -1.1045007 , -2.2974925 ,
         0.8641774 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 437 image.
[array([[-0.49948138, -1.4559898 , -1.8299623 , -1.2334476 , -0.15546355,
        -0.30659193,  0.19717628,  0.381956  , -1.1045022 , -2.2974887 ,
         0.8641722 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 438 image.
[array([[-0.4994843 , -1.4559932 , -1.829964  , -1.2334325 , -0.15546334,
        -0.3066072 ,  0.19716734,  0.3819541 , -1.104511  , -2.2974799 ,
         0.8641685 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 439 image.
[array([[-0.49948472, -1.4559915 , -1.829965  , -1.2334404 , -0.15545839,
        -0.30660552,  0.1971679 ,  0.38195586, -1.104513  , -2.2974925 ,
         0.8641728 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 440 image.
[array([[-0.49948704, -1.4559871 , -1.829947

[array([[-0.49949744, -1.4559929 , -1.8299441 , -1.2334394 , -0.15545383,
        -0.3065884 ,  0.19716996,  0.38196567, -1.1044918 , -2.2975035 ,
         0.86417055]], dtype=float32), array([10], dtype=int64)]
It`s processing 473 image.
[array([[-0.4994988 , -1.4559937 , -1.8299414 , -1.233439  , -0.15545264,
        -0.30658603,  0.19716877,  0.38196564, -1.1044915 , -2.2975035 ,
         0.86417025]], dtype=float32), array([10], dtype=int64)]
It`s processing 474 image.
[array([[-0.49950176, -1.4559919 , -1.8299403 , -1.2334388 , -0.15545025,
        -0.3065827 ,  0.19717017,  0.3819672 , -1.1044881 , -2.2975025 ,
         0.8641716 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 475 image.
[array([[-0.4994984 , -1.4559947 , -1.829941  , -1.2334388 , -0.1554541 ,
        -0.30658498,  0.19716936,  0.3819659 , -1.1044906 , -2.2975025 ,
         0.8641697 ]], dtype=float32), array([10], dtype=int64)]
It`s processing 476 image.
[array([[-0.4995016 , -1.455993  , -1.829939

KeyboardInterrupt: 